<a href="https://colab.research.google.com/github/markub3327/CarRacing-RNN/blob/master/CarRacing_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/markub3327/CarRacing-RNN.git
%cd CarRacing-RNN/
%ls

Cloning into 'CarRacing-RNN'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 598 (delta 30), reused 34 (delta 9), pack-reused 537
Receiving objects: 100% (598/598), 480.11 MiB | 16.45 MiB/s, done.
Resolving deltas: 100% (297/297), done.
/content/CarRacing-RNN
data/     main.py    nets/        README.md*  utils/
LICENSE*  model.png  preview.gif  save/


In [6]:
import os
import numpy as np

from nets.agent import Agent

def Training(datasets='data/', hid=[32, 64]):
    
    def load_datasets():
        # the list of datasets
        dat = []
        
        # scan datasets
        with os.scandir(datasets) as entries:
            for entry in entries:
                dat.append(entry.path)
        
        return dat

    def read_dataset(path):
        with np.load(path) as data:
            f = data['frames']
            a = data['actions']
            print(f.shape)
            print(a.shape)

            # make timesteps
            f, a = make_timesteps(f, a)
            print(f.shape)
            print(a.shape)

            # shuffle dataset after loading from file
            f, a = shuffle_dataset(f, a)
        
        return f, a

    def make_timesteps(f_dat, a_dat, num_frames=4):
        # generate random indexes
        rand_idxs = np.arange(num_frames + 1, f_dat.shape[0], dtype=np.int)
        print(rand_idxs)
        print(rand_idxs.shape)

        states = np.zeros((rand_idxs.shape[0], num_frames) + f_dat.shape[1:], dtype=np.uint8)
        #next_states = np.zeros((BATCH_SIZE, POST_PROCESS_IMAGE_SIZE[0], POST_PROCESS_IMAGE_SIZE[1], NUM_FRAMES), dtype=np.float32)
            
        for i, idx in enumerate(rand_idxs):
            states[i] = f_dat[idx-num_frames-1:idx-1]
            #next_states[i] = self._frames[idx-num_frames:idx]

        return states, a_dat[rand_idxs] #, self._rewards[rand_idxs], next_states, self._terminal[rand_idxs]
        
    def shuffle_dataset(f_dat, a_dat):
        idx = np.arange(0, f_dat.shape[0], dtype=np.int)
        print(idx)
        np.random.shuffle(idx)
        print(idx)

        return f_dat[idx], a_dat[idx]


    # create network
    agent = Agent()
    agent.create((4, 96, 96, 3), hid=hid)

    # save model's plot
    agent.save_plot()
    
    # load datasets from folder
    datasets = load_datasets()

    # take every dataset from folder
    for dat in datasets:
        print('+-----------------------------------------------+')

        print(f"Loading data from dataset: {dat}")
        f, a = read_dataset(dat)

        print('Run training...')
        print('|-----------------------------------------------|')
        agent.train(f, a, epochs=1000, top_only=True)
        
        # re-shuffle dataset before fine-tuning
        f, a = shuffle_dataset(f, a)

        print('Run fine-tuning...')
        print('|-----------------------------------------------|')
        agent.train(f, a, epochs=1000, top_only=False)

        print('+-----------------------------------------------+')

    # save model
    agent.save('/content/drive/My Drive/my.h5')
  
Training()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


9412608/9406464 [==============================] - 1s 0us/step
+-----------------------------------------------+
Loading data from dataset: data/data_part1 (14020).npz
(14020, 96, 96, 3)
(14020, 3)
[    5     6     7 ... 14017 14018 14019]
(14015,)
(14015, 4, 96, 96, 3)
(14015, 3)
[    0     1     2 ... 14012 14013 14014]
[11792  9148  6214 ...  7486  7408 10304]
Run training...
|-----------------------------------------------|
Model: "agent"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        [(None, 4, 96, 96, 3 0                                            
__________________________________________________________________________________________________
normalization (TimeDistributed) (None, 4, 96, 96, 3) 0           image_input[0][0]                
__________________________________________________________

wandb: Network error resolved after 0:00:11.441382, resuming normal operation.


136/136 [==============================] - 47s 345ms/step - loss: 0.0582 - steering_loss: 0.0482 - acceleration_brake_loss: 0.0100 - val_loss: 0.0315 - val_steering_loss: 0.0249 - val_acceleration_brake_loss: 0.0067
Epoch 2/1000
136/136 [==============================] - 45s 328ms/step - loss: 0.0505 - steering_loss: 0.0419 - acceleration_brake_loss: 0.0086 - val_loss: 0.0334 - val_steering_loss: 0.0267 - val_acceleration_brake_loss: 0.0067
Epoch 3/1000
136/136 [==============================] - 45s 328ms/step - loss: 0.0473 - steering_loss: 0.0385 - acceleration_brake_loss: 0.0088 - val_loss: 0.0330 - val_steering_loss: 0.0268 - val_acceleration_brake_loss: 0.0061
Epoch 4/1000
136/136 [==============================] - 45s 328ms/step - loss: 0.0424 - steering_loss: 0.0346 - acceleration_brake_loss: 0.0078 - val_loss: 0.0327 - val_steering_loss: 0.0266 - val_acceleration_brake_loss: 0.0061
Epoch 5/1000
136/136 [==============================] - 45s 328ms/step - loss: 0.0395 - steering_